# Imports

In [190]:
import pandas as pd
from rent_warehouse_mania_objects import street_synonyms, city_names
from dlt.sources.helpers import requests
from bs4 import BeautifulSoup
from hashlib import md5
import re

# Teste scrappers

## Chaves na mão

### Variáveis iniciais

In [30]:
url: str = "https://www.chavesnamao.com.br/imoveis-para-alugar/pr-curitiba/"
rent_html_class: str = "imoveis__Card-obm8pe-0 tNifl"

### Funções

In [226]:
# Fazer função de filtrar lista de palavras
def filter_words(words_list, desired_c=None, not_desired_c=None):
    # Retornar lista com as palavras contenham todos os desired_c e nenhum dos not_desired_c
    if desired_c and not_desired_c:
        return [word for word in words_list if any(char in word for char in desired_c) and all(char not in word for char in not_desired_c)]
    
    # Retornar lista com as palavras contenham todos os desired_c
    elif desired_c:
        return [word for word in words_list if any(char in word for char in desired_c)]

    # Retornar lista com as palavras não contenham not_desired_c
    elif not_desired_c:
        return [word for word in words_list if all(char not in word for char in not_desired_c)]

# Fazer função de pegar o possível preço do imóvel em uma lista
def get_rent_price(possible_prices: list, max_rent: float = float("inf")) -> float:
    # Fazer lista vazia para guardar os valores que realmente podem ser o preço
    new_possible_prices = []

    # Iterar as possíveis strings que contem o preço
    for price in possible_prices:
        # Se a string iterada tiver qualquer dígito numérico
        if any(l.isdigit() for l in price):
            # Mantenha apenas os numéricos ou ".", "," na string
            price = "".join([l for l in price if l.isdigit() or l in (".", ",")])

            # Em casos de preço com virgula, pegue a primeira parte do preço apenas
            price = price.split(",")[0]

            # Remova . se eles forem começo ou final de string
            price = price.strip(".")

            # Converta . em _
            price = price.replace(".", "_")
            
            # Tente
            try:
                # Conversão para floar
                price = float(price)

                # Se o preço for maior que 100_000, divida-o por esse valor
                if price >= 100_000:
                    price = price / 100_000

                # Guarde ela na lista 
                new_possible_prices.append(price)

            # Em caso de erro de tipo na conversão, pule essa string
            except ValueError:
                print(f"String -> {price} não é um preço de imóvel!")

    # Filtrar alugueis abaixo de max_rent
    new_possible_prices = [price for price in new_possible_prices if price < max_rent]

    # Retorne o maior campo da lista
    return max(new_possible_prices)

# Fazer função de pegar o tamanho do imóvel
def get_rent_size(possible_sizes: list,  max_size: int = float("inf")) -> float:
    # Fazer lista vazia para guardar os valores que realmente podem ser o tamanho do imóvel
    new_possible_sizes = []

    # Iterar as possíveis strings que contem o tamanho
    for size in possible_sizes:
        # Se a string iterada tiver qualquer dígito numérico que não seja ², ³
        if any(l.isdigit() and l != "²" and l != "³" for l in size):
            # Deixe somente numeros e não deixe ², ³
            size = "".join([l for l in size if (l.isdigit() and l != "²" and l != "³") or (l == ",")])

            # Divida o tamanho na "," (se tiver) e pegue o primeiro campo
            size = size.split(",")[0]

            # Tente
            try:
                # Conversão para inteiro
                size = int(size)
            
                # Guarde ela na lista
                new_possible_sizes.append(size)

            # Em caso de erro de tipo na conversão, pule essa string
            except ValueError:
                print(f"String -> {size} não é um tamanho de imóvel!")

    # Filtrar tamanhnos abaixo de max_size
    new_possible_sizes = [size for size in new_possible_sizes if size < max_size]

    # Retorne o primeiro campo da lista
    return new_possible_sizes[0]

# Fazer função de pegar o endereço do imóvel
def get_rent_adress(rent_splited_words: list) -> str:
    # Faça uma lista vazia para guardar os possíveis indexes
    possible_rua_index = []
    possible_city_index = []

    # Iterar todas as palavras buscando os possíveis indices da rua
    for iword, word in enumerate(rent_splited_words):
        # Se a palavra estiver na lista de sinonimos para rua, guarde esse index
        if any(i in word.lower() for i in street_synonyms):
            possible_rua_index.append(iword)

        # Se a palavra estiver na lista de cidades, guarde esse index
        if any(i in word.lower() for i in city_names):
            possible_city_index.append(iword)

    # Se ambas as listas tiverem idnex
    if len(possible_rua_index) != 0 and len(possible_city_index) != 0:
        # Retornar o join de todas as palavras na string, escolhendo o min(possible_rua_index) até o max(possible_city_index)
        return " ".join(rent_splited_words[min(possible_rua_index):max(possible_city_index)])
    
    # Do contrário
    else:
        # Retorne um join geral
        return " ".join(rent_splited_words)


### Scrapper

In [227]:
# Pegar a response
response = requests.get(url)

# Se o status vier 200, prossiga
if response.status_code == 200:
    # Pegar sopa de letras com o BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Pegar todas as divs com a classe rent_html_class
    imoveis = [imovel.text for imovel in soup.find_all("div", class_=rent_html_class)]

    di = {
        "id": [],
        "preco": [],
        "tamanho": [],
        "endereco": [],
    }
    
    # Iterar todos os imóveis
    for imovel in imoveis:
        # Splitar palavras do card do imovel
        imovel_words = imovel.split()

        # Pegar campo de preço do imovel
        preco = get_rent_price(filter_words(imovel_words, desired_c=[".", "$"], not_desired_c=["²", "³"]), max_rent=50_000)

        # Pegar campo de tamanho
        tamanho = get_rent_size(filter_words(imovel_words, desired_c=["²", "³"], not_desired_c="$"), max_size=5_000)

        # Pegar campo de endereço
        endereco = get_rent_adress(imovel_words)

        # Gerar id com hash md5
        rent_id = md5(endereco.encode("utf-8")).hexdigest()

        # apendaar na lista
        di["id"].append(rent_id)
        di["endereco"].append(endereco)
        di["preco"].append(preco)
        di["tamanho"].append(tamanho)

In [228]:
pd.DataFrame(di)

,id,preco,tamanho,endereco
0,d037e5de7d3da0c93c4c9dfe5f2e1235,4250.0,115,"curitiba/prAvenida São José, 752 Cristo Rei,"
1,9c9837bfa2c966106f93b31cce2fad3e,2000.0,80,"banheirosRua João Dembinski, 2512 Campo Compri..."
2,7673ffb9db43aa04d39c642b4b5a8a87,277.0,31,curitiba/prAvenida Coronel Francisco Heráclito...
3,dd920d705a23f75e6edd011e64dfe38f,3000.0,30,"curitiba/prRua Affife Mansur, 33 Novo Mundo,"
4,8c0083ceb7ca93804317948820bb22dc,1862.0,928,"curitiba.Avenida Presidente Affonso Camargo, 8..."
5,c51df604f357c30711b5789135694c9d,1580.0,19,"Studio com 1 dormitório para alugar, 19 m² por..."
6,e051e5326642e311627581474aeb60cc,26288.0,329,"curitiba/prRua Martim Afonso Bigorrilho,"
7,fc99d1b902a65360348dff78ef279e02,16789.0,233,curitiba/prAvenida Cândido de Abreu Centro Cív...
8,b38a3d5944f013a425a3c882bc104975,11600.0,2000,"xaxim!Rua Leôncio Derosso, 75 Xaxim,"
9,f568ff26c6dd9ad3f41b7b225b693c79,4200.0,180,"rua david tows, 701, xaxim, curitiba, 180 m2 p..."


In [214]:
# Definir o primeiro index como 0 e o final como len(rent_splited_words) por padrão
rua_index = 0
city_index = len(imovel_words) - 1

# Setar status como não definido
rua_index_defined = False
city_index_defined = False

# Iterar todas as palavras
for iword, word in enumerate(imovel_words):
    # Se a palavra estiver na lista de sinonimos para rua, guarde esse index
    if any(i in word.lower() for i in street_synonyms) and rua_index_defined == False:
        rua_index = iword
        rua_index_defined = True

    # Se a palavra estiver na lista de cidades, guarde esse index
    elif any(i in word.lower() for i in city_names) and city_index_defined == False:
        city_index = iword
        city_index_defined = True

In [215]:
city_index

13

In [216]:
rua_index

26

In [218]:
imovel_words[city_index]

'curitiba/prSala'

In [220]:
imovel_words[rua_index]

'curitiba/prAvenida'

In [221]:
imovel_words

['Sala',
 'para',
 'alugar,',
 '31',
 'm²',
 'por',
 'r$',
 '800/mês',
 '-',
 'jardim',
 'das',
 'américas',
 '-',
 'curitiba/prSala',
 'para',
 'alugar,',
 '31',
 'm²',
 'por',
 'r$',
 '800/mês',
 '-',
 'jardim',
 'das',
 'américas',
 '-',
 'curitiba/prAvenida',
 'Coronel',
 'Francisco',
 'Heráclito',
 'dos',
 'Santos,',
 '765',
 'Jardim',
 'das',
 'Américas,',
 'CuritibaR$',
 '800',
 'Cond.:',
 'R$',
 '115Sala',
 'Comercial',
 'para',
 'locação',
 'na',
 'Avenida',
 'Coronel',
 'Francisco',
 'Heráclito',
 'dos',
 'Santos',
 '-',
 'Bairro',
 'Jardim',
 'das',
 'Américas',
 'Seu',
 'novo',
 'empreendimento',
 'na',
 'melhor',
 'localização',
 'do',
 'Jardim',
 'das',
 'Américas!',
 'Ótima',
 'sala',
 'comercial',
 'para',
 'qualquer',
 'ramo',
 'de',
 'atividade',
 'com',
 '31m²,',
 'contendo:',
 '01',
 'sala,',
 'lavabo',
 'e',
 'uma',
 'vaga',
 'de',
 'garagem',
 'rotativa.',
 'Conjunto',
 'possui',
 'baixa',
 'taxa',
 'e',
 'disponibiliza',
 'interfone.',
 'A',
 'rede',
 'de',
 'ene